# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [36]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [37]:
import csv
vacation_df = pd.read_csv ("../WeatherPy/city_weather_data.csv")
vacation_df.dropna(inplace = True) 
vacation_df.head()


,Unnamed: 0,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ambilobe,-13.20,49.05,68.11,82,32,4.16,MG,1592872024
1,1,Lucea,18.45,-78.17,86.00,74,20,14.99,JM,1592872024
2,2,Saint-Pierre,-21.34,55.48,66.20,77,19,5.82,RE,1592872024
3,3,Atuona,-9.80,-139.03,79.83,79,43,13.73,PF,1592872024
4,4,Codrington,-38.27,141.97,58.24,85,93,16.37,AU,1592872024


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = vacation_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
rating = vacation_df["Humidity"].astype(float)

figure_layout = {
    'width': '800px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=20,
                                 point_radius=1.5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
#Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

cleaned_vacation_df = vacation_df.loc[(vacation_df["Max Temp"]<80) & (vacation_df["Max Temp"]>60) 
                                      & (vacation_df["Wind Speed"]>10)  & (vacation_df["Cloudiness"]==0) & (vacation_df["Humidity"] != 65) ]
cleaned_vacation_df = cleaned_vacation_df.dropna()
cleaned_vacation_df
del cleaned_vacation_df['Unnamed: 0']
cleaned_vacation_df = cleaned_vacation_df.reset_index(drop=True)
cleaned_vacation_df

,City Name,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mount Isa,-20.73,139.50,60.80,35,0,13.87,AU,1592871778
1,Srednekolymsk,67.45,153.68,67.51,26,0,16.51,RU,1592872037
2,San Roque,36.21,-5.38,69.80,73,0,11.41,ES,1592872051
3,São Filipe,14.90,-24.50,77.94,72,0,13.53,CV,1592872060
4,Charters Towers,-20.10,146.27,63.07,33,0,13.33,AU,1592872060
5,Kumul,42.80,93.45,71.33,20,0,12.71,CN,1592872062
6,Kununurra,-15.77,128.73,69.80,24,0,10.29,AU,1592872067


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = cleaned_vacation_df[["City Name", "Country", "Latitude","Longitude"]].copy()
hotel_df["Hotel Name"]=""
hotel_df

,City Name,Country,Latitude,Longitude,Hotel Name
0,Mount Isa,AU,-20.73,139.50,
1,Srednekolymsk,RU,67.45,153.68,
2,San Roque,ES,36.21,-5.38,
3,São Filipe,CV,14.90,-24.50,
4,Charters Towers,AU,-20.10,146.27,
5,Kumul,CN,42.80,93.45,
6,Kununurra,AU,-15.77,128.73,


In [39]:
# geocoordinates

target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
for index, row  in hotel_df.iterrows():
    #get the lat and long of the data frame
    lat = row["Latitude"]
    long = row["Longitude"]
    params["location"] = f"{lat},{long}"
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    places_data = response.json()
    # Get the hotel from the response
    try:
        hotel_df.loc[index,"Hotel Name"] =places_data["results"][0]["name"]
    except(KeyError, IndexError):
        print("No Results, get AirBnB")

    # Print the json (pretty printed)
   # print(json.dumps(places_data, indent=4, sort_keys=True))
hotel_df

No Results, get AirBnB


,City Name,Country,Latitude,Longitude,Hotel Name
0,Mount Isa,AU,-20.73,139.50,ibis Styles Mt Isa Verona
1,Srednekolymsk,RU,67.45,153.68,
2,San Roque,ES,36.21,-5.38,Hotel Guadacorte Park
3,São Filipe,CV,14.90,-24.50,Tortuga B&B
4,Charters Towers,AU,-20.10,146.27,Cattleman's Rest Motor Inn
5,Kumul,CN,42.80,93.45,Laoyutang Resort
6,Kununurra,AU,-15.77,128.73,Hotel Kununurra


In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [43]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add layer
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…